In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/dreamgaussian

!pip install -q torch-ema einops tensorboardX plyfile dearpygui huggingface_hub diffusers==0.21.4 accelerate transformers xatlas
!pip install -q trimesh PyMCubes pymeshlab rembg[gpu,cli] omegaconf ninja
%cd /content/dreamgaussian
!git clone --recursive https://github.com/ashawkey/diff-gaussian-rasterization
!pip install -q ./diff-gaussian-rasterization
!pip install -q ./simple-knn
!pip install -q https://github.com/camenduru/wheels/releases/download/colab/nvdiffrast-0.3.1-py3-none-any.whl
!pip install -q git+https://github.com/ashawkey/kiuikit

/content
Cloning into 'dreamgaussian'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 67 (delta 21), reused 13 (delta 13), pack-reused 24
Receiving objects: 100% (67/67), 206.51 KiB | 6.07 MiB/s, done.
Resolving deltas: 100% (22/22), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.7/694.7 kB 8.1 MB/s eta 0:00:00
     ━━━

In [ ]:
NAME="tank"
IMAGE="tank.png"
IMAGE_PROCESSED="tank_rgba.png"

In [ ]:
%run process.py data/{IMAGE}

[INFO] loading image data/tank.png...
[INFO] background removal...
[INFO] recenter...


In [ ]:
#@title training!

# stage 1
%run main.py --config configs/image.yaml input=data/{IMAGE_PROCESSED} save_path={NAME} elevation=0 force_cuda_rast=True

# stage 2
%run main2.py --config configs/image.yaml input=data/{IMAGE_PROCESSED} save_path={NAME} elevation=0 force_cuda_rast=True

[INFO] load image from data/tank_rgba.png...
Number of points at initialisation :  5000
[INFO] loading zero123...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

[INFO] loaded zero123!


100%|██████████| 500/500 [01:13<00:00,  6.83it/s]


[INFO] save model to logs/tank_model.ply.


...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

[INFO] loaded zero123!


100%|██████████| 50/50 [01:01<00:00,  1.23s/it]


[INFO] save model to logs/tank.obj.


In [ ]:
#@title render a video for displaying

# the final mesh is saved to ./logs/NAME.obj
%run -m kiui.render logs/{NAME}.obj --save_video {NAME}.mp4 --wogui --force_cuda_rast

from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width=450):
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

show_video(f'{NAME}.mp4')

[INFO] load obj mesh: use texture from: logs/tank_albedo.png
[INFO] load obj mesh: load texture: (1024, 1024, 3)
[INFO] load mesh, v: torch.Size([15649, 3]), f: torch.Size([31298, 3])
[INFO] load mesh, vn: torch.Size([15649, 3]), fn: torch.Size([31298, 3])
[INFO] load mesh, vt: torch.Size([18364, 2]), ft: torch.Size([31298, 3])


100%|██████████| 1/1 [00:00<00:00,  1.24it/s]
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
